In [ ]:
import numpy as np
import pandas as pd
import re
import sklearn
import spacy
import nltk

In [ ]:
#Import training data 
#allData = pd.read_csv("remove_past_bin.txt", delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

In [ ]:
# Text preprocessing and adds speaker column
speakers = []
for i in range(len(allData)):
    if (':' in allData.iloc[i, 0]):
        index = allData.iloc[i, 0].index(':')
        speakers.append((allData.iloc[i, 0])[:index+1])
        allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0]) 
    else:
        speakers.append('F:')
    allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0])
allData.insert (0, "speaker", speakers)

In [ ]:
class Sentence:
# Creates a Sentence object
# text = text of Sentence
# rp_bin = which occurence of been or bin is being looked at in the sentence
  # eg. if the variable be is set to 2, the model will observe whether the second noun/verb clause contains an instance of remote past bin
# num = the index of the Sentence in the original csv
    remoteBin = 0
    r1 = 0; r2 = 0;
    def __init__(self, text, rp, num, true):
        self.text = text
        self.num = num
        self.rp = rp
        self.true = true

In [ ]:
# Preproccesses training text for linguistic analysis

lines = []

with open("remote_past_bin.txt", "r", encoding="utf-8") as tsv_file: #these files are tab separated but you can easily change this to CSV
    tsv_reader = csv.reader(tsv_file, delimiter="\t")
    header = next(tsv_reader)
    for row in tsv_reader:
        row = row[:1]
        row[0] = re.sub("[a-zA-Z]:\s+", "", row[0]) #removes the interviewer tag
        row[0] = re.sub("\s{2,}", " ", row[0]) #removes excessive spaces
        row[0] = re.sub("’|‘", "'", row[0]) #fixes apostrophes
        row[0] = re.sub("—", "--", row[0]) #m-dash was causing formatting issues, changed it to two dashes
        row[0] = re.sub("“", '"', row[0]) #fixes quotation marks    
        lines.append(row[0])
         
print(len(lines))

In [ ]:
# Create an array of Sentence objects with at least one 'been' or 'bin'

sen = []
for line in range(len(lines)):
        rp = 1
        numBin = 0
        parsed = nlp(lines[line])
        for i, word in enumerate(parsed):
            if (word.text.lower() == "been" or word.text.lower() == "bin"):
                numBin += 1
        if (numBin >= 1):
            sen.append(Sentence(lines[line], rp, line))
            while (numBin > 1):
                rp += 1
                sen.append(Sentence(lines[line], rp, line))
                numBin -= 1
for Sentence in range(len(sen)):
    sen[Sentence].text = (sen[Sentence].text).replace('"','')

In [ ]:
# Tags the Sentence objects with each rule

index = -1
senNum = 1
numBin = 0
binIndices = []

for Sentence in sen:
    index += 1 
    numBin = 0
    if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
        senNum += 1
    else:
        senNum = 1
    parsed = nlp(Sentence.text)
    for i, word in enumerate(parsed):
        if (i >= 0 and (word.text.lower() in ('bin', 'been')) and Sentence.remoteBin != 1):
            numBin += 1
            if (numBin == senNum):
                w.append(word.pos_)
                binIndices.append(i)
                binChildren = list(word.children)
                if(word.head == word):
                    binSibling = []
                else:
                    binSibling = list(word.head.children)
                binDep = word.dep_
                binPOS = word.pos_
                binIndex = i
                num = 1
                next = None
                prev = None
                if (prev.pos_ == 'NOUN' or prev.pos_ == 'PRON' or prev.pos_ == 'PROPN') and (next.pos_ == 'VERB' or next.pos_ == 'AUX')
                    r1 = 1
#                if (Sentence.r2 != 1):
#                    Sentence.r2 = 0
                if (Sentence.r1 == 1):# and Sentence.r2 != 1):
                    Sentence.remoteBin = 1
                else:
                    Sentence.remoteBin = -1

In [ ]:
# Consolidation bin values of duplicated sentences
# e.g. if any have disagreement, assign all bin values to 1, else keep them as 0

multBininSentence = []

for i in range(len(sen)-1):
    if (i < len(sen)-1):
        j = i+1 
        if (sen[i].text != sen[j].text):
            if (sen[i].rp == 1):
                multBininSentence.append(True)
            else:
                multBininSentence.append(False)
        while (sen[i].text == sen[j].text):
            if (sen[i].rp == 1 or sen[j].rp == 1):
                multBininSentence.append(True)
            else:
                multBininSentence.append(False)
            if (sen[j].rp == 1):
                sen[i].rp == sen[j].rp
            if (sen[i].rp == 1):
                sen[j].rp == sen[i].rp
            if (j < len(sen)-1):
                j +=1
            else:
                break

if len(multBininSentence) != len(sen):
    if (sen[-1].rp == 1):
        multBininSentence.append(True)
    else:
        multBininSentence.append(False)

In [ ]:
# Goes back to the original data and annotates each sentences with remote past bin or not

remoteBin = []
for i in range(len(allData)):
    m = 0
    for j in range(len(sen)):
        if i == sen[j].num:
            if (m != 1):
                m = sen[j].rp
            if (m == 1 and ("%RPB" not in allData.iloc[i, allData.columns.get_loc('Speaker')])):
                allData.loc[i, 'Speaker'] = allData.iloc[i, allData.columns.get_loc('Speaker')] + " %RPB"
    remoteBin.append(m)
allData['remotePastBin'] = remoteBin

In [ ]:
# Print classification report

predictions = allData['remotePastBin']
y = allData['remotePastBIN']
target_names = ['remotePast', 'none']
print(classification_report(y, predictions, target_names=target_names))

In [ ]:
with open('pastBin.txt', 'w+', newline='') as file:
   allData.to_csv('pastBin.txt', sep = '\t')